In [1]:
import warnings
warnings.filterwarnings('ignore')
import pyterrier as pt
if not pt.started():
    pt.init()

import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', False)

import ir_measures
from ir_measures import * # imports all supported measures, e.g., AP, nDCG, RR, P

# import pyt_splade
# splade = pyt_splade.Splade()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_28207/3663056234.py:5: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
# dataset_name = 'msmarco-passage'
# dataset = pt.get_dataset(f'irds:{dataset_name}')

# eval_dataset = pt.get_dataset(f'irds:{dataset_name}/dev')
# topics = eval_dataset.get_topics()
# qrels = eval_dataset.get_qrels()

import ir_datasets
eval = ir_datasets.load("msmarco-passage/dev")
topics = pd.DataFrame(eval.queries_iter())
qrels = pd.DataFrame(eval.qrels_iter())


In [3]:
topics.shape

(101093, 2)

In [ ]:
# df.shape

In [ ]:
# import ir_datasets
# dataset = ir_datasets.load("msmarco-passage/dev")
# count = 0
# for doc in dataset.docs_iter():
#     print(doc)
#     count +=1
#     if count >5:
#         break

In [ ]:
# ms_docids = {key['docno']:0 for key in dataset.get_corpus_iter(verbose=True)}

In [ ]:
import json

def save(dict, file):
    with open(file,'w') as f:
        json.dump(dict, f)


In [ ]:
def load(file):
    with open(file,'r') as f:
        ms_docids = json.load(f)
    return ms_docids

In [ ]:
# savedisk(ms_docids, './ms_docids.json')

In [ ]:
# for k,v in ms_docids.items():
#     print(type(k),type(v))

In [ ]:
#################### bm25 ###########################

In [ ]:

def calc_rtr_score(df, docid_dict, savename):
    for query_id in pt.tqdm(topics['query_id']):
        D = df[df['qid'] == np.int64(query_id)]
        for dno in D['docno']:
            rank =  D[D['docno'] == dno]['rank'].values[0]
            score = 100/np.log(rank + 2) # plus 2 because the ranks start from zero
            docid_dict[str(dno)] += score
    
    save(docid_dict,savename)   


In [ ]:
def calc_metrics(modelname):
    scoredF = f'./results/{modelname}_docids_100.json'
    if not os.path.exists(scoredF):
        csv = f'/nfs/datasets/cxj/retrievability-bias/results_{modelname}_100.csv'
        init_msmarco_dict = './results/ms_docids.json'
        docids_score = load(init_msmarco_dict)
        for query_id in pt.tqdm(topics['query_id']):
            D = df[df['qid'] == np.int64(query_id)]
            for dno in D['docno']:
                rank =  D[D['docno'] == dno]['rank'].values[0]
                score = 100/np.log(rank + 2) # plus 2 because the ranks start from zero
                docids_score[str(dno)] += score

    docids_score = load(scoredF)
    scores_df = pd.DataFrame.from_dict(docids_score, orient="index",columns=["score"])
    scores_df = scores_df[scores_df["score"]>0]
    scores = scores_df['score'].to_list()
    calc_stats(scores)
    gini_value = G(scores)


In [ ]:
def transform_df(df):
    df_run = df.rename(columns={'qid':'query_id','docid':'doc_id'})
    del df_run['score']
    df_run['score'] = 10-df_run['rank']
    df_run[['query_id','doc_id']] = df_run[['query_id','doc_id']].astype(str)

    return df_run

In [ ]:
calc_metrics('bm25')

In [ ]:
bm25 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_100.csv')

In [ ]:
bm25_docids_100 = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(bm25, bm25_docids_100, './results/bm25_docids_100.json')

In [ ]:
monot5 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_monot5_100.csv') # 100 docs retrieved for each query.

In [ ]:
monot5_docids_100 = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(monot5, monot5_docids_100, './results/monot5_docids_100.json')

In [ ]:
################ splade  ##################################

In [ ]:
splade = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_splade_100.csv') # 100 docs retrieved for each query.

In [ ]:
splade.head(2)

In [ ]:
splade_docids_100 = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(splade, splade_docids_100, './results/splade_docids_100.json')

In [ ]:
colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_colbert_100.csv')

In [ ]:
colbert['docno'] = colbert['docid']

In [ ]:
colbert.head(2)

In [ ]:
colbert_docids_100 = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(colbert, colbert_docids_100, './results/colbert_docids_100.json')

In [ ]:
bm25_colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_colbert_100.csv')
bm25_colbert.info()

In [ ]:
bm25_colbert_docids_100 = load('./results/ms_docids.json')
calc_rtr_score(bm25_colbert, bm25_colbert_docids_100, './results/bm25_colbert_docids_100.json')


In [ ]:
bm25_docids = load('./results/bm25_docids_100.json')

In [ ]:
monot5_docids = load('./results/monot5_docids_100.json')

In [ ]:
splade_docids = load('./results/splade_docids_100.json')

In [ ]:
colbert_docids = load('./results/colbert_docids_100.json')

In [ ]:
bm25_colbert_docids = load('./results/bm25_colbert_docids_100.json')

In [ ]:
bm25_csv = pd.DataFrame.from_dict(bm25_docids, orient="index",columns=["score"])
bm25_csv.to_csv('./results/bm25_100.csv',index=False)

In [ ]:
monot5_csv = pd.DataFrame.from_dict(monot5_docids, orient="index",columns=["score"])
monot5_csv.to_csv('./results/monot5_100.csv',index=False)

In [ ]:
splade_csv = pd.DataFrame.from_dict(splade_docids, orient="index",columns=["score"])
splade_csv.to_csv('./results/splade_100.csv',index=False)


In [ ]:
colbert_csv = pd.DataFrame.from_dict(colbert_docids, orient="index",columns=["score"])
colbert_csv.to_csv('./results/colbert_100.csv',index=False)

In [ ]:
bm25_colbert_csv = pd.DataFrame.from_dict(bm25_colbert_docids, orient="index",columns=["score"])
bm25_colbert_csv.to_csv('./results/bm25_colbert_100.csv',index=False)

In [ ]:
bm25_csv = pd.read_csv('./results/bm25_100.csv')
bm25_csv = bm25_csv[bm25_csv["score"]>0]
bm25_csv.to_csv('./results/bm25_csv_100_pruned.csv', index=False)

In [ ]:


monot5_csv = pd.read_csv('./results/monot5_100.csv')
monot5_csv = monot5_csv[monot5_csv["score"]>0]
monot5_csv.to_csv('./results/monot5_csv_100_pruned.csv', index=False)



In [ ]:
splade_csv = pd.read_csv('./results/splade_100.csv')
splade_csv = splade_csv[splade_csv["score"]>0]
splade_csv.to_csv('./results/splade_csv_100_pruned.csv', index=False)



In [ ]:
colbert_csv = pd.read_csv('./results/colbert_100.csv')
colbert_csv = colbert_csv[colbert_csv["score"]>0]
colbert_csv.to_csv('./results/colbert_csv_100_pruned.csv', index=False)

In [ ]:
bm25_colbert_csv = pd.read_csv('./results/bm25_colbert_100.csv')
bm25_colbert_csv = bm25_colbert_csv[bm25_colbert_csv["score"]>0]
bm25_colbert_csv.to_csv('./results/bm25_colbert_csv_100_pruned.csv', index=False)

In [ ]:
# bm25_csv = pd.read_csv('./results/bm25.csv')
# column = bm25_csv.columns[1]
# bm25_csv = bm25_csv[bm25_csv[column] == 0]
# bm25_csv.to_csv('./results/bm25_csv_with_0.csv', index=False)

# monot5_csv = pd.read_csv('./results/monot5.csv')
# column = monot5_csv.columns[1]
# monot5_csv = monot5_csv[monot5_csv[column] == 0]
# monot5_csv.to_csv('./results/monot5_csv_with_0.csv', index=False)

# splade_csv = pd.read_csv('./results/splade.csv')
# column = splade_csv.columns[1]
# splade_csv = splade_csv[splade_csv[column] == 0]
# splade_csv.to_csv('./results/splade_csv_with_0.csv', index=False)

# colbert_csv = pd.read_csv('./results/colbert.csv')
# column = colbert_csv.columns[1]
# colbert_csv = colbert_csv[colbert_csv[column] == 0]
# colbert_csv.to_csv('./results/colbert_csv_with_0.csv', index=False)

In [ ]:
bm25_csv.shape

In [ ]:
monot5_csv.shape

In [ ]:
splade_csv.shape

In [ ]:
colbert_csv.shape

In [ ]:
bm25_colbert_csv.shape

In [ ]:
def G(v):
    v = np.array(v)
    bins = np.linspace(0., 100., 11)
    total = float(np.sum(v))
    yvals = [0]
    for b in bins[1:]:
        bin_vals = v[v <= np.percentile(v, b)]
        bin_fraction = (np.sum(bin_vals) / total) * 100.0
        yvals.append(bin_fraction)
    # perfect equality area
    pe_area = np.trapz(bins, x=bins)
    # lorenz area
    lorenz_area = np.trapz(yvals, x=bins)
    gini_val = (pe_area - lorenz_area) / float(pe_area)
    # return bins, yvals, gini_val
    print('gini:', gini_val)

In [ ]:
import statistics

def calc_stats(values):
    # values = list(data.values())

    mean = statistics.mean(values)
    std_dev = statistics.stdev(values) 
    print(f"Mean: {mean}")
    print(f"Standard Deviation: {std_dev}")


In [ ]:
bm25_scores = bm25_csv['score'].to_list()
calc_stats(bm25_scores)

In [ ]:
gini_value = G(bm25_scores)
gini_value

In [ ]:
monot5_scores = monot5_csv['score'].to_list()
calc_stats(monot5_scores)

In [ ]:
gini_value = G(monot5_scores)
gini_value

In [ ]:
splade_scores = splade_csv['score'].to_list()
calc_stats(splade_scores)

In [ ]:
gini_value = G(splade_scores)
gini_value

In [ ]:
colbert_scores = colbert_csv['score'].to_list()
calc_stats(colbert_scores)

In [ ]:
gini_value = G(colbert_scores)
gini_value

In [ ]:
bm25_colbert_scores = bm25_colbert_csv['score'].to_list()
calc_stats(bm25_colbert_scores)

In [ ]:
gini_value = G(bm25_colbert_scores)
gini_value

In [ ]:
bm25 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_100.csv')
bm25 = transform_df(bm25)
ir_measures.calc_aggregate([nDCG@10, RR], qrels, bm25)

In [ ]:
monot5 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_monot5_100.csv')

In [ ]:
splade = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_splade_100.csv')

In [ ]:
colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_colbert_100.csv')

In [ ]:
bm25_colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_colbert_100.csv')
bm25_colbert['docid'] = bm25_colbert['docno']
bm25_colbert = transform_df(bm25_colbert)
ir_measures.calc_aggregate([nDCG@10, RR], qrels, bm25_colbert)